In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('league_disqualifications.csv', header=[0,1])

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df = df.fillna(0)

In [3]:
first_name = ""
new_columns = []
first = True
for name in df.columns:
    if first:
        first = False
        new_col = tuple([first_name, name[1]])
    elif "Unnamed" in name[0]:
        new_col = tuple([first_name, name[1]])
    else:
        first_name = name[0]
        new_col = name
    new_columns.append(new_col)
df.columns = new_columns

In [4]:
district = ('', 'District')
league = ('', 'League')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
def generate_new_df(df, district, league):
    sums = df.select_dtypes(np.number).sum().rename('totals')
    l = []
    div = league
    n = ''
    v = ''
    dis = district
    cur = []
    first = True
    second = True
    total = 0
    for name, val in sums.index:
        if first: 
            first = False
        else:
            if name == n:
                cur.append(sums[(name,val)])
                total += sums[(name,val)]
            else:
                if not second:
                    cur.append(total)
                    cur.append(df.shape[0])
                    l.append(cur)
                    total = 0
                else:
                    second = False
                cur = [dis, div, name]
                n = name
                cur.append(sums[(name,val)])
                total += sums[(name,val)]
                                       
    new_df = pd.DataFrame(l, columns =['District', 'League', 'Sport','1 Gm','2 Gm','Tour','1 Yr','Coach','Total','NumSchools'])
    new_df = new_df.loc[new_df['Total'] != 0]
    return new_df

result = pd.DataFrame()
for i in range(1,10):
    cur_dis = i
    cur_district = df.loc[df[district] == cur_dis]
    for lg in cur_district[league].unique():
        cur_df = cur_district.loc[cur_district[league] == lg]
        created = generate_new_df(cur_df, cur_dis, lg)
        result = pd.concat([result, created], ignore_index=True)
        
result['DQ per School'] = result.apply(lambda row: row['Total'] / row['NumSchools'], axis=1)
result = result.loc[result['Sport'] != 'TOTALS']
print(result)

     District                          League                Sport  1 Gm  \
0           1                  Pioneer Valley   BASKETBALL-JV BOYS   0.0   
1           1                  Pioneer Valley    BASKETBALL-V BOYS   1.0   
2           1                  Pioneer Valley   BASKETBALL-V GIRLS   0.0   
3           1                  Pioneer Valley           FOOTBALL-V   1.0   
4           1                  Pioneer Valley    Ice Hockey-V BOYS   3.0   
5           1                  Pioneer Valley       SOCCER-JV BOYS   2.0   
6           1                  Pioneer Valley        SOCCER-V BOYS  14.0   
7           1                  Pioneer Valley       SOCCER-V GIRLS   2.0   
8           1                  Pioneer Valley       SWIMMING-GIRLS   1.0   
10          1                Berkshire County   BASKETBALL-JV BOYS   1.0   
11          1                Berkshire County           FOOTBALL-V   0.0   
12          1                Berkshire County        SOCCER-V BOYS   6.0   
13          

In [6]:
result.to_csv('disqualifications.csv')

# Use above to save to a csv file. Need to make further edits for now.

In [ ]:
# result = result.loc[result['Sport'] != 'TOTALS']
# by_total = result.sort_values(by='Total', axis=0, ascending=False, ignore_index=True)
# by_total.to_csv('dqs_by_total.csv')

# Use above to remove rows labeled TOTALS, sort the values based on the column 'Total', and write to a .csv file.

In [ ]:
# by_total = result.sort_values(by='Total', axis=0, ascending=False, ignore_index=True)

In [ ]:
# sports = result['Sport'].unique()
# by_sport = pd.DataFrame()
# for s in sports:
#     current = result.loc[result['Sport'] == s]
#     current = current.sort_values(by='Total', axis=0, ascending=False, ignore_index=True)
#     by_sport = pd.concat([by_sport, current], ignore_index=True)

In [ ]:
## for each unique sport, separate out the dataframe, sort by Total, and then recombine to one full dataframe
# by_sport.to_csv('dqs_by_sport.csv')

In [ ]:
# dqs_by_numSchools = result.sort_values(by='DQ per School', axis=0, ascending=False, ignore_index=True)
# dqs_by_numSchools.to_csv('dqs_per_school.csv')

In [ ]:
sports = result['Sport'].unique()
by_sport_df = pd.DataFrame()
for s in sports:
    current_sport = result.loc[result['Sport'] == s]
    ordered_district_df = pd.DataFrame()
    districts = result['District'].unique()
    for d in districts:
        cur = current_sport.loc[current_sport['District'] == d]
        cur = cur.sort_values(by='League', axis=0, ascending=True, ignore_index=True)
        ordered_district_df = pd.concat([ordered_district_df, cur], ignore_index=True)
    by_sport_df = pd.concat([by_sport_df, ordered_district_df], ignore_index=True)
print(by_sport_df)
# add the totals and make .csv, then you're done pending some Excel formatting.

In [ ]:
sports = by_sport_df['Sport'].unique()
by_sport_with_totals = pd.DataFrame()
for s in sports:
    current_sport = by_sport_df.loc[by_sport_df['Sport'] == s]
    current_sport.loc['TOTAL'] = current_sport.sum()
    current_sport.at['TOTAL','District'] = 'T'
    current_sport.at['TOTAL','League'] = 'TOTAL'
    current_sport.at['TOTAL','Sport'] = s
    current_sport.at['TOTAL','NumSchools'] = '-'
    current_sport.at['TOTAL','DQ per School'] = '-'
    by_sport_with_totals = pd.concat([by_sport_with_totals, current_sport], ignore_index=True)
print(by_sport_with_totals)
by_sport_with_totals.to_csv('dqs_by_sport_with_totals.csv')